In [64]:
import pandas as pd
import geopandas as gpd
file = 'train.csv'
trip_df = []
for chunk in pd.read_csv(file, chunksize=10000):
    trip_df.append(chunk)
trip_df = pd.concat(trip_df, axis='rows')

In [27]:
trip_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
dtypes: float64(4), int64(3), object(4)
memory usage: 133.5+ MB


In [28]:
print(trip_df.head())

          id  vendor_id      pickup_datetime     dropoff_datetime  \
0  id2875421          2  2016-03-14 17:24:55  2016-03-14 17:32:30   
1  id2377394          1  2016-06-12 00:43:35  2016-06-12 00:54:38   
2  id3858529          2  2016-01-19 11:35:24  2016-01-19 12:10:48   
3  id3504673          2  2016-04-06 19:32:31  2016-04-06 19:39:40   
4  id2181028          2  2016-03-26 13:30:55  2016-03-26 13:38:10   

   passenger_count  pickup_longitude  pickup_latitude  dropoff_longitude  \
0                1        -73.982155        40.767937         -73.964630   
1                1        -73.980415        40.738564         -73.999481   
2                1        -73.979027        40.763939         -74.005333   
3                1        -74.010040        40.719971         -74.012268   
4                1        -73.973053        40.793209         -73.972923   

   dropoff_latitude store_and_fwd_flag  trip_duration  
0         40.765602                  N            455  
1         40.731

In [29]:
#Slice dates from pickup datetime column to create new date column
dates = []
for i in trip_df['pickup_datetime']:
    dates.append(i[:10])
trip_df['date'] = pd.to_datetime(dates)

In [30]:
#Convert vendor ID from Int64 to Categorical
trip_df['vendor_id'] = trip_df['vendor_id'].astype('category')
#Pickup and drop off should be in Datetime format
trip_df['pickup_datetime'] = pd.to_datetime(trip_df['pickup_datetime'])
trip_df['dropoff_datetime'] = pd.to_datetime(trip_df['dropoff_datetime'])

In [36]:
#Import weather data
file2 = 'NYCweather2016.csv'
weather = pd.read_csv(file2)
print(weather.info())
print(weather.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2375 entries, 0 to 2374
Data columns (total 20 columns):
STATION      2375 non-null object
NAME         2375 non-null object
LATITUDE     2375 non-null float64
LONGITUDE    2375 non-null float64
ELEVATION    2375 non-null float64
DATE         2375 non-null object
AWND         1088 non-null float64
PRCP         2372 non-null float64
SNOW         1871 non-null float64
SNWD         1132 non-null float64
TAVG         732 non-null float64
TMAX         1098 non-null float64
TMIN         1098 non-null float64
WT01         353 non-null float64
WT02         33 non-null float64
WT03         44 non-null float64
WT04         7 non-null float64
WT06         7 non-null float64
WT08         197 non-null float64
WT09         4 non-null float64
dtypes: float64(17), object(3)
memory usage: 371.2+ KB
None
       STATION                         NAME  LATITUDE  LONGITUDE  ELEVATION  \
0  USW00094728  NY CITY CENTRAL PARK, NY US  40.77898  -73.96925       42

In [37]:
weather['DATE'] = pd.to_datetime(weather['DATE'])

In [38]:
#Review data for each weather station
for name in weather['NAME'].drop_duplicates():
    print('-----------------------------------------------------', name, sep='\n')
    print(weather.loc[weather['NAME'] == name].info())

-----------------------------------------------------
NY CITY CENTRAL PARK, NY US
<class 'pandas.core.frame.DataFrame'>
Int64Index: 366 entries, 0 to 365
Data columns (total 20 columns):
STATION      366 non-null object
NAME         366 non-null object
LATITUDE     366 non-null float64
LONGITUDE    366 non-null float64
ELEVATION    366 non-null float64
DATE         366 non-null datetime64[ns]
AWND         356 non-null float64
PRCP         366 non-null float64
SNOW         366 non-null float64
SNWD         366 non-null float64
TAVG         0 non-null float64
TMAX         366 non-null float64
TMIN         366 non-null float64
WT01         131 non-null float64
WT02         6 non-null float64
WT03         0 non-null float64
WT04         1 non-null float64
WT06         2 non-null float64
WT08         99 non-null float64
WT09         0 non-null float64
dtypes: datetime64[ns](1), float64(17), object(2)
memory usage: 60.0+ KB
None
-----------------------------------------------------
MIDDLE VI

In [39]:
#Use only weather events recorded at LaGuardia Airport due to highest availability of data and most central location.
weather = weather.loc[weather['NAME'] == 'LA GUARDIA AIRPORT, NY US']

In [40]:
#Merge dataframes
nyc_taxi = pd.merge(trip_df, weather, how='left', left_on='date', right_on = 'DATE')

In [41]:
nyc_taxi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1458644 entries, 0 to 1458643
Data columns (total 32 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null category
pickup_datetime       1458644 non-null datetime64[ns]
dropoff_datetime      1458644 non-null datetime64[ns]
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
date                  1458644 non-null datetime64[ns]
STATION               1458644 non-null object
NAME                  1458644 non-null object
LATITUDE              1458644 non-null float64
LONGITUDE             1458644 non-null float64
ELEVATION             1458644 non-null float64
DATE                  1458644 non-null datetime64[ns]
AWND                

In [45]:
nyc_taxi.iloc[:20, -7:]

,WT01,WT02,WT03,WT04,WT06,WT08,WT09
0,1.0,NaN,NaN,NaN,NaN,1.0,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,1.0,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
#Columns representing weather types (WT01 - WT09) have 1's indicating yes and NaN indicating no. Replace NaN with 0.
bools = {1: True, 0: False}
for col in nyc_taxi.iloc[:,-7:]:
    nyc_taxi[col] = nyc_taxi[col].fillna(value=0)
    nyc_taxi[col] = nyc_taxi[col].map(bools)

In [47]:
print(nyc_taxi.iloc[:20,-7:])

     WT01   WT02   WT03   WT04   WT06   WT08   WT09
0    True  False  False  False  False   True  False
1   False  False  False  False  False  False  False
2   False  False  False  False  False  False  False
3   False  False  False  False  False  False  False
4   False  False  False  False  False  False  False
5   False  False  False  False  False  False  False
6   False  False  False  False  False  False  False
7    True  False  False  False  False  False  False
8   False  False  False  False  False   True  False
9   False  False  False  False  False  False  False
10  False  False  False  False  False  False  False
11  False  False  False  False  False  False  False
12  False  False  False  False  False  False  False
13  False  False  False  False  False  False  False
14  False  False  False  False  False   True  False
15  False  False  False  False  False   True  False
16  False  False  False  False  False  False  False
17  False  False  False  False  False  False  False
18   True  F

In [48]:
for col in nyc_taxi:
    print(col)

id
vendor_id
pickup_datetime
dropoff_datetime
passenger_count
pickup_longitude
pickup_latitude
dropoff_longitude
dropoff_latitude
store_and_fwd_flag
trip_duration
date
STATION
NAME
LATITUDE
LONGITUDE
ELEVATION
DATE
AWND
PRCP
SNOW
SNWD
TAVG
TMAX
TMIN
WT01
WT02
WT03
WT04
WT06
WT08
WT09


In [49]:
c = ['vendor_id','store_and_fwd_flag','ELEVATION','DATE']
for i in c:
    del nyc_taxi[i]

In [50]:
columns = ['id','pickup_datetime','dropoff_datetime','passenger_count','pickup_longitude','pickup_latitude','dropoff_longitude',
          'dropoff_latitude','trip_duration','date','station_num','station_name','station_latitude','station_longitude','average_wind',
          'precipitation','snowfall','snow_depth','avg_temp','max_temp','min_temp','fog','fog_heavy','thunder','ice_pellets','glaze',
          'smoke_haze','blowing_snow']

In [51]:
nyc_taxi.columns = columns

In [52]:
nyc_taxi[:1000].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 28 columns):
id                   1000 non-null object
pickup_datetime      1000 non-null datetime64[ns]
dropoff_datetime     1000 non-null datetime64[ns]
passenger_count      1000 non-null int64
pickup_longitude     1000 non-null float64
pickup_latitude      1000 non-null float64
dropoff_longitude    1000 non-null float64
dropoff_latitude     1000 non-null float64
trip_duration        1000 non-null int64
date                 1000 non-null datetime64[ns]
station_num          1000 non-null object
station_name         1000 non-null object
station_latitude     1000 non-null float64
station_longitude    1000 non-null float64
average_wind         1000 non-null float64
precipitation        1000 non-null float64
snowfall             1000 non-null float64
snow_depth           1000 non-null float64
avg_temp             1000 non-null float64
max_temp             1000 non-null float64
min_temp            

In [53]:
#Remove column that stores duplicate index
nyc_taxi = nyc_taxi.iloc[:,1:]

#Set index to datetime of pickup
nyc_taxi['pickup_datetime'] = pd.to_datetime(nyc_taxi['pickup_datetime'])
datetime_ind = list(nyc_taxi['pickup_datetime'])
nyc_taxi.index = datetime_ind
#Set drop off to datetime
nyc_taxi['dropoff_datetime'] = pd.to_datetime(nyc_taxi['dropoff_datetime'])

#While we're at it, pair the datetimes with their respective days of the week
day_of_week = [x.weekday() for x in datetime_ind]
nyc_taxi['day_of_week'] = day_of_week

#Create a separate column with the names of the weekdays in case they're needed later
weekday_dict = {0: 'Monday',
               1: 'Tuesday',
               2: 'Wednesday',
               3: 'Thursday',
               4: 'Friday',
               5: 'Saturday',
               6: 'Sunday'}
nyc_taxi['weekday_name'] = nyc_taxi['day_of_week'].map(weekday_dict)

In [54]:
#Change days of week to categoricals
nyc_taxi['day_of_week'] = nyc_taxi['day_of_week'].astype('category').cat.as_ordered()
nyc_taxi['weekday_name'] = nyc_taxi['weekday_name'].astype('category')

In [65]:
#Write the taxi/weather data to a csv
nyc_taxi.to_csv('nyc_taxi.csv')

In [68]:
#Read in geospatial data for New York
ny_state = gpd.read_file('ZillowNeighborhoods-NY.shp')

In [69]:
#Locate duplicate regions
ny_state[ny_state['RegionID'].duplicated()]

,State,County,City,Name,RegionID,geometry
246,NY,New York,New York,Marble Hill,270880,POLYGON ((-73.91071526499991 40.87890143900006...
282,NY,Kings,New York,Vinegar Hill,270960,POLYGON ((-73.97901955399993 40.70666396400009...
310,NY,Bronx,New York,Rikers Island,270922,"POLYGON ((-73.8865870059999 40.79891837000008,..."
375,NY,New York,New York,Brooklyn Heights,403122,POLYGON ((-73.99610507599969 40.70422567500025...


In [70]:
#Investigate one of the duplicate regions
ny_state[ny_state['RegionID'] == '270960']

,State,County,City,Name,RegionID,geometry
281,NY,New York,New York,Vinegar Hill,270960,POLYGON ((-73.97901955399993 40.70666396400009...
282,NY,Kings,New York,Vinegar Hill,270960,POLYGON ((-73.97901955399993 40.70666396400009...


In [71]:
#Drop duplicate regions
ny_state = ny_state.loc[ny_state['RegionID'].drop_duplicates().index]

#Check that all regions are unique (output should be 1)
ny_state['RegionID'].nunique(dropna = True) / len(ny_state['RegionID'])

1.0

In [72]:
#Set index to region ID
ny_state['RegionID'] = ny_state['RegionID'].astype('int')
ny_state = ny_state.set_index(['RegionID'])
ny_state = ny_state.sort_index()

In [73]:
#All data come from the same state, drop state column
ny_state = ny_state.drop('State', axis=1)

In [74]:
#List columns of ny_state Dataframe
columns = list(ny_state.select_dtypes(include=['object']).columns.values)
columns.remove('geometry')
print(columns)

['County', 'City', 'Name']


In [76]:
ny_state['County'] = ny_state['County'].astype('category')

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 575 entries, 3698 to 764316
Data columns (total 4 columns):
County      575 non-null object
City        575 non-null object
Name        575 non-null object
geometry    575 non-null object
dtypes: object(4)
memory usage: 22.5+ KB


In [77]:
print(ny_state.info(verbose=False))

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 575 entries, 3698 to 764316
Columns: 4 entries, County to geometry
dtypes: object(4)
memory usage: 22.5+ KB
None


In [78]:
ny_state.drop('geometry', axis=1).describe()

,County,City,Name
count,575,575,575
unique,24,50,558
top,Queens,New York,Downtown
freq,70,274,5


In [79]:
nyc = ny_state.loc[ny_state['City'] == 'New York']
nyc.shape

(274, 4)

In [84]:
nyc.to_file('nyc_geo.shp')